In [1]:
"""
Created on Tue Mar 16 19:11:35 2021

@author: y.Xing
"""

import numpy as np
import pandas as pd
import requests

## Load the dataa
"""
url_central_london = "https://cycling.data.tfl.gov.uk/CycleCountsProgramme/Central%20London%20(area).xlsx"
url_inner_london = "https://cycling.data.tfl.gov.uk/CycleCountsProgramme/Inner%20London%20(area).xlsx"
url_outer_london = "https://cycling.data.tfl.gov.uk/CycleCountsProgramme/Outer%20London%20(area).xlsx"

def get_count_data(url:str, dest:str):
    resp = requests.get(url)
    with open('./'+dest + '.xlsx', 'wb') as output:
        output.write(resp.content)
        
get_count_data(url_central_london, "central_london")
get_count_data(url_inner_london, "inner_london")
get_count_data(url_outer_london, "outer_london")

def excel_to_csv(file_name:str):
    df = pd.read_excel("./"+file_name + ".xlsx", sheet_name = 1)
    df.to_csv(file_name + '.csv', sep='\t', index = False)
    return df

central_london = excel_to_csv("central_london")
inner_london = excel_to_csv("inner_london")
outer_london = excel_to_csv("outer_london")
"""
# Data cleaning
# central_london.csv
central_london = pd.read_csv("central_london.csv")
count_sites_list = pd.read_csv("count_sites_list.csv", usecols=("UnqID", "Borough"))

In [2]:
# modeling
import matplotlib.pyplot as plt
import statsmodels.api as sm
from patsy import dmatrices

# Create a pandas DataFrame for the counts data set.
df = pd.merge(
    central_london, count_sites_list, "left", left_on="Site ID", right_on="UnqID"
)
pick_agg_cols = {
    "Number of normal cycles": ["mean"],
    "Number of cycle hire bikes": ["mean"],
    "Total cycles": ["mean"],
}
df_grouped = (
    df.groupby(["Borough", "Equivalent financial quarter", "Period", "Weather"])
    .agg(pick_agg_cols)
    .reset_index()
)


# Add a few derived regression variables.

# Create the training and testing data sets.
mask = np.random.rand(len(df_grouped)) < 0.8
df_train = df_grouped[mask]
df_test = df_grouped[~mask]
print("Training data set length=" + str(len(df_train)))
print("Testing data set length=" + str(len(df_test)))

# Setup the regression expression in patsy notation. We are telling patsy that BB_COUNT is our dependent variable and
# it depends on the regression variables: DAY, DAY_OF_WEEK, MONTH, HIGH_T, LOW_T and PRECIP.
expr_train = """df_train['Total cycles'] ~ df_train['Borough'] + df_train['Equivalent financial quarter'] + df_train['Period'] + df_train['Weather']"""
expr_test = """df_test['Total cycles'] ~ df_test['Borough'] + df_test['Equivalent financial quarter'] + df_test['Period'] + df_test['Weather']"""

# Set up the X and y matrices
y_train, X_train = dmatrices(expr_train, df_train, return_type="dataframe")
y_test, X_test = dmatrices(expr_test, df_test, return_type="dataframe")

# Using the statsmodels GLM class, train the Poisson regression model on the training data set.
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()

# Print the training summary.
print(poisson_training_results.summary())

# Make some predictions on the test data set.
poisson_predictions = poisson_training_results.get_prediction(X_test)
# .summary_frame() returns a pandas DataFrame
predictions_summary_frame = poisson_predictions.summary_frame()
print(predictions_summary_frame)

predicted_counts = predictions_summary_frame["mean"]
actual_counts = y_test["Total cycles"]

# Mlot the predicted counts versus the actual counts for the test data.
fig = plt.figure()
fig.suptitle("Predicted versus actual total cycling counts")
(predicted,) = plt.plot(X_test.index, predicted_counts, "go-", label="Predicted counts")
(actual,) = plt.plot(X_test.index, actual_counts, "ro-", label="Actual counts")
plt.legend(handles=[predicted, actual])
plt.show()

# Show scatter plot of Actual versus Predicted counts
plt.clf()
fig = plt.figure()
fig.suptitle("Scatter plot of Actual versus Predicted counts")
plt.scatter(x=predicted_counts, y=actual_counts, marker=".")
plt.xlabel("Predicted counts")
plt.ylabel("Actual counts")
plt.show()

Training data set length=2804
Testing data set length=692
                    Generalized Linear Model Regression Results                     
Dep. Variable:     df_train['Total cycles']   No. Observations:                 2804
Model:                                  GLM   Df Residuals:                     2510
Model Family:                       Poisson   Df Model:                          293
Link Function:                          log   Scale:                          1.0000
Method:                                IRLS   Log-Likelihood:                -11117.
Date:                      Tue, 16 Mar 2021   Deviance:                       10738.
Time:                              23:13:44   Pearson chi2:                 1.17e+04
No. Iterations:                          23                                         
Covariance Type:                  nonrobust                                         
                                                             coef    std err          z     

ValueError: shapes (692,155) and (294,) not aligned: 155 (dim 1) != 294 (dim 0)